# 1. Download the dataset

In [1]:
import os
import sys
print(os.getcwd())
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../../")) 

sys.path.append(project_root)  # Ensure src/ is found
sys.path.append(os.path.abspath(os.getcwd()))

sys.path

d:\BK University\Semester 242\Machine Learning\ML_capybara\src\models\pca_features


['C:\\Users\\Dell\\.pyenv\\pyenv-win\\versions\\3.9.12\\python39.zip',
 'C:\\Users\\Dell\\.pyenv\\pyenv-win\\versions\\3.9.12\\DLLs',
 'C:\\Users\\Dell\\.pyenv\\pyenv-win\\versions\\3.9.12\\lib',
 'C:\\Users\\Dell\\.pyenv\\pyenv-win\\versions\\3.9.12',
 'd:\\BK University\\Semester 242\\Machine Learning\\ML_capybara\\.venv',
 '',
 'd:\\BK University\\Semester 242\\Machine Learning\\ML_capybara\\.venv\\lib\\site-packages',
 'C:\\Users\\Dell\\.pyenv\\pyenv-win\\versions\\3.9.12\\lib\\site-packages',
 'C:\\Users\\Dell\\.pyenv\\pyenv-win\\versions\\3.9.12\\lib\\site-packages\\win32',
 'C:\\Users\\Dell\\.pyenv\\pyenv-win\\versions\\3.9.12\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\Dell\\.pyenv\\pyenv-win\\versions\\3.9.12\\lib\\site-packages\\Pythonwin',
 'd:\\BK University\\Semester 242\\Machine Learning\\ML_capybara',
 'd:\\BK University\\Semester 242\\Machine Learning\\ML_capybara\\src\\models\\pca_features']

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

current_dir = Path().resolve().parent

# Navigate two levels up to the project root and access data/test.csv
path = current_dir.parent.parent / "data"

for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

D:\BK University\Semester 242\Machine Learning\ML_capybara\data\fasttext_subwords_300.kv
D:\BK University\Semester 242\Machine Learning\ML_capybara\data\fasttext_subwords_300.kv.vectors.npy
D:\BK University\Semester 242\Machine Learning\ML_capybara\data\test.csv
D:\BK University\Semester 242\Machine Learning\ML_capybara\data\train.csv


## Download library

In [3]:
from sklearn.metrics import f1_score
from src.data import Preprocessing

In [4]:
data_path = f"{path}/train.csv"
test_path = f"{path}/test.csv"

train_df = pd.read_csv(data_path, encoding='latin-1', usecols=['text', 'sentiment'])
test_df = pd.read_csv(test_path, encoding='latin-1', usecols=['text', 'sentiment'])
train_df = train_df.dropna()
test_df = test_df.dropna()

train_df.info(), test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27480 entries, 0 to 27480
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       27480 non-null  object
 1   sentiment  27480 non-null  object
dtypes: object(2)
memory usage: 644.1+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 3534 entries, 0 to 3533
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       3534 non-null   object
 1   sentiment  3534 non-null   object
dtypes: object(2)
memory usage: 82.8+ KB


(None, None)

## Preprocessing

In [5]:
# Download NLTK resources (if not already downloaded)
prepro = Preprocessing()

train_df['token'] = train_df['text'].apply(prepro.tokenize_preprocessing)
test_df['token'] = test_df['text'].apply(prepro.tokenize_preprocessing)
max_len = max(train_df['token'].apply(lambda x: len(x)))
train_df['token'], max_len

✓ 'punkt' is already available.
✓ 'stopwords' is already available.
✗ 'wordnet' not found. Downloading...


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


✓ 'averaged_perceptron_tagger' is already available.


(0                                   [id, responded, going]
 1                            [sooo, sad, miss, san, diego]
 2                                          [bos, bullying]
 3                                [interview, leave, alone]
 4            [son, couldnt, put, release, already, bought]
                                ...                        
 27476    [wish, could, come, see, u, denver, husband, l...
 27477    [ive, wondered, rake, client, made, clear, net...
 27478    [yay, good, enjoy, break, probably, need, hect...
 27479                                              [worth]
 27480              [flirting, going, atg, smile, yay, hug]
 Name: token, Length: 27480, dtype: object,
 23)

## Download `Word2Vec` Model

In [6]:
from word_embedding import WordEmbeddingVectors

model = WordEmbeddingVectors()

X_train_feature = model.vectorize(train_df['token'])
X_test_feature = model.vectorize(test_df['token'])

D:\BK University\Semester 242\Machine Learning\ML_capybara\data\fasttext_subwords_300.kv
D:\BK University\Semester 242\Machine Learning\ML_capybara\data\fasttext_subwords_300.kv.vectors.npy
D:\BK University\Semester 242\Machine Learning\ML_capybara\data\test.csv
D:\BK University\Semester 242\Machine Learning\ML_capybara\data\train.csv
Loader model successfully ! Vector size: 300


# 2. Testing on model

## 2.1 Logistic Regression

In [7]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train_feature, train_df['sentiment'])

# Predict and evaluate
preds = clf.predict(X_test_feature)
acc_score = f1_score(test_df['sentiment'], preds, average="macro")
print("F1 score:", acc_score)

F1 score: 0.6704800519277726


## 2.2 Bayesian Model

In [8]:
from sklearn.naive_bayes import GaussianNB

nbmodel = GaussianNB()
nbmodel.fit(X_train_feature, train_df['sentiment'])

GaussianNB()

In [9]:
# Predict and evaluate
preds = nbmodel.predict(X_test_feature)
acc_score = f1_score(test_df['sentiment'], preds, average="macro")
print("Accuracy:", acc_score)

Accuracy: 0.5331825951023416


# 3. Using PCA

## 3.1 Using PCA with fixed component

In [10]:
# sys.path.append(os.path.abspath(os.getcwd()))  # Ensure src/ is found
from pca_feature import PCA

pca = PCA()
# pca.fit_by_fix_features(X_train_feature, top_k=20)
pca.fit_by_amount_of_variance(X_train_feature, threshold=0.8)
X_train_pca = pca.transform(X_train_feature)
X_test_pca = pca.transform(X_test_feature)
X_train_pca.shape

PCA keep 87 components correponding to keep 0.801655049277397 of total variance


(27480, 87)

### 3.1.1 Testing on Bayesian model

In [11]:
nbmodel = GaussianNB()
nbmodel.fit(X_train_pca, train_df['sentiment'])

preds = nbmodel.predict(X_test_pca)
acc_score = f1_score(test_df['sentiment'], preds, average="macro")
print("Accuracy:", acc_score)

Accuracy: 0.4605212052992777


### 3.1.2 Testing on Logistic model

In [12]:
clf = LogisticRegression()
clf.fit(X_train_pca, train_df['sentiment'])

# Predict and evaluate
preds = clf.predict(X_test_pca)
acc_score = f1_score(test_df['sentiment'], preds, average="macro")
print("F1 score:", acc_score)

F1 score: 0.6556082732906992


## 3.2 Using PCA with fixed threshold of variance

In [13]:
pca = PCA()
pca.fit_by_amount_of_variance(X_train_feature, threshold=0.8)
X_train_pca = pca.transform(X_train_feature)
X_test_pca = pca.transform(X_test_feature)
X_train_pca.shape

PCA keep 87 components correponding to keep 0.801655049277397 of total variance


(27480, 87)

### 3.2.1 Testing on NB model

In [14]:
nbmodel = GaussianNB()
nbmodel.fit(X_train_pca, train_df['sentiment'])

preds = nbmodel.predict(X_test_pca)
acc_score = f1_score(test_df['sentiment'], preds, average="macro")
print("Accuracy:", acc_score)

Accuracy: 0.46006178144527504


### 3.2.2 Testing on Logistic Model

In [15]:
clf = LogisticRegression()
clf.fit(X_train_pca, train_df['sentiment'])

# Predict and evaluate
preds = clf.predict(X_test_pca)
acc_score = f1_score(test_df['sentiment'], preds, average="macro")
print("F1 score:", acc_score)

F1 score: 0.65761781789225
